In [1]:
import numpy as np
import scipy.integrate
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pylab as plt
%matplotlib inline

In [2]:
def lorenz(t, x, b, sig, rho):
    
    return [sig * (x[1] - x[0]),  rho * x[0] - x[0]*x[2] - x[1], x[0]*x[1] - b*x[2]]

In [3]:
# Simulate Lorenz system
dt = 0.01 
T = 8  
t = np.arange(0, T, dt)
b = 8/3
sig = 10

In [4]:
def get_trajectories(rho=10):
    
    x0 = 30*(np.random.uniform(0, 1, 3)-0.5)
    y = scipy.integrate.odeint(func = lambda t,x: lorenz(t, x, b, sig, rho), y0=x0, t=t, rtol=1e-10, atol=1e-11, tfirst=True)
        
    inputs = y[:-1,:]
    outputs = y[1:,:]
    
    for i in range(100):
        
        x0 = 30*(np.random.uniform(0, 1, 3)-0.5)
        y = scipy.integrate.odeint(func = lambda t,x: lorenz(t, x, b, sig, rho), y0=x0, t=t, rtol=1e-10, atol=1e-11, tfirst=True)
        
        inputs = np.vstack((inputs, y[:-1, :]))
        outputs = np.vstack((outputs, y[1:, :]))

    
    return inputs, outputs

In [5]:
inputs_10, outputs_10 = get_trajectories(10)
inputs_28, outputs_28 = get_trajectories(28)
inputs_40, outputs_40 = get_trajectories(40)

In [6]:
inputs = np.vstack((inputs_10, inputs_28, inputs_40))

In [7]:
outputs = np.vstack((outputs_10, outputs_28, outputs_40))

In [8]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(3, activation='linear'))
model.compile(loss='mse', optimizer='adam', validation_split=0.2)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [9]:
model.fit(inputs, outputs, epochs=100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
242097/242097 [==============================] - 5s 23us/step - loss: 0.4022
Epoch 2/100
242097/242097 [==============================] - 5s 21us/step - loss: 0.0336
Epoch 3/100
242097/242097 [==============================] - 5s 22us/step - loss: 0.0241
Epoch 4/100
242097/242097 [==============================] - 5s 22us/step - loss: 0.0202
Epoch 5/100
242097/242097 [==============================] - 5s 21us/step - loss: 0.0181
Epoch 6/100
242097/242097 [==============================] - 5s 21us/step - loss: 0.0168
Epoch 7/100
242097/242097 [==============================] - 5s 22us/step - loss: 0.0157
Epoch 8/100
242097/242097 [==============================] - 5s 22us/step - loss: 0.0153
Epoch 9/100
242097/242097 [==============================] - 5s 21us/step - loss: 0.0143
Epoch 10/100
242097/242097 [==============================] - 5s 21us/step - loss: 0.0144
Epoch 11/100
242097/242097 [==============================] -

242097/242097 [==============================] - 5s 21us/step - loss: 0.0081
Epoch 89/100
242097/242097 [==============================] - 5s 21us/step - loss: 0.0081
Epoch 90/100
242097/242097 [==============================] - 5s 21us/step - loss: 0.0080
Epoch 91/100
242097/242097 [==============================] - 5s 21us/step - loss: 0.0080
Epoch 92/100
242097/242097 [==============================] - 5s 22us/step - loss: 0.0081
Epoch 93/100
242097/242097 [==============================] - 5s 21us/step - loss: 0.0081
Epoch 94/100
242097/242097 [==============================] - 5s 21us/step - loss: 0.0081
Epoch 95/100
242097/242097 [==============================] - 5s 21us/step - loss: 0.0080
Epoch 96/100
242097/242097 [==============================] - 5s 21us/step - loss: 0.0080
Epoch 97/100
242097/242097 [==============================] - 5s 21us/step - loss: 0.0081
Epoch 98/100
242097/242097 [==============================] - 5s 21us/step - loss: 0.0079
Epoch 99/100
242097/242

In [61]:
def test(rho, model):   

    x0_test = 30*(np.random.uniform(0, 1, 3)-0.5)

    y_test = scipy.integrate.odeint(func = lambda t,x: lorenz(t, x, b, sig, rho), y0=x0_test, t=t, rtol=1e-10, atol=1e-11, tfirst=True)

    y_pred = np.zeros(y_test.shape)
    y_pred[0, :] = x0_test
    
    for i in range(1, y_pred.shape[0]):
        y_pred[i,:] = model.predict(np.expand_dims(y_pred[i-1, :], axis=1).T)
        
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot3D(y_test[:,0], y_test[:,1], y_test[:,2])
    ax.set_title('Original')
    ax.plot3D(y_pred[:,0], y_pred[:,1], y_pred[:,2])
    ax.set_title('Predicted')
    plt.title('NN rho = {}'.format(rho))
    
    
    plt.figure()
    plt.plot(y_test[:,0])
    plt.plot(y_pred[:,0])
    plt.legend(['ODE45', 'NN'])
    plt.xlabel('t')
    plt.ylabel('x(t)')
    
    plt.figure()
    plt.plot(y_test[:,1])
    plt.plot(y_pred[:,1])
    plt.legend(['ODE45', 'NN'])
    plt.xlabel('t')
    plt.ylabel('y(t)')
    
    plt.figure()
    plt.plot(y_test[:,2])
    plt.plot(y_pred[:,2])
    plt.legend(['ODE45', 'NN'])
    plt.xlabel('t')
    plt.ylabel('z(t)')